In [1]:
import pandas as pd
import geopandas as gpd
import plotly as plt
from pathlib import Path
import rasterio
from rasterstats import zonal_stats


In [2]:
root_dir = Path.cwd().resolve().parent
data_dir = root_dir / "data"

caminho_geotiff = data_dir / "raster_html.tif"
caminho_favelas = data_dir / "all_favelas_sp_2022.gpkg"

In [3]:
gdf_favelas = gpd.read_file(caminho_favelas)

with rasterio.open(caminho_geotiff) as src:
    raster_crs = src.crs
    raster_nodata = src.nodata

# projeta favelas pro CRS do raster (pra calcular média sem distorção)
if gdf_favelas.crs != raster_crs:
    gdf_favelas = gdf_favelas.to_crs(raster_crs)

# ---------- média por polígono ----------
stats = zonal_stats(
    vectors=gdf_favelas.geometry,
    raster=str(caminho_geotiff),
    stats=["mean"],
    nodata=raster_nodata,
    all_touched=False
)
gdf_favelas["temp_media"] = [s["mean"] if s["mean"] is not None else float('nan') for s in stats]

In [4]:
gdf_favelas.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 1359 entries, 0 to 1358
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   cd_fcu      1359 non-null   object  
 1   nm_fcu      1359 non-null   object  
 2   cd_uf       1359 non-null   object  
 3   nm_uf       1359 non-null   object  
 4   sigla_uf    1359 non-null   object  
 5   cd_mun      1359 non-null   object  
 6   Pop_2022    1359 non-null   int64   
 7   Area        1359 non-null   int64   
 8   Density     1359 non-null   int64   
 9   LST         1359 non-null   float64 
 10  geometry    1359 non-null   geometry
 11  temp_media  1358 non-null   float64 
dtypes: float64(2), geometry(1), int64(3), object(6)
memory usage: 127.5+ KB


In [5]:
gdf_favelas.head(3)

,cd_fcu,nm_fcu,cd_uf,nm_uf,sigla_uf,cd_mun,Pop_2022,Area,Density,LST,geometry,temp_media
0,35503080069,Jardim Miliunas,35,São Paulo,SP,3550308,1159,25712,0,0.0,"MULTIPOLYGON (((-46.37165 -23.50024, -46.3717 ...",43.443211
1,35503082222,Avenida Arraias do Araguaia,35,São Paulo,SP,3550308,333,10204,0,0.0,"MULTIPOLYGON (((-46.50249 -23.5805, -46.50288 ...",37.426762
2,35503081646,Pavanas,35,São Paulo,SP,3550308,161,2968,0,0.0,"MULTIPOLYGON (((-46.64777 -23.66911, -46.6481 ...",39.712868


In [11]:
# %% [markdown]
# Top 10 favelas por população + temperatura média (cores do QML)
# - Usa gdf_favelas já carregado (colunas: nm_fcu, Pop_2022, temp_media)
# - Lê a rampa de cores do QML em data/mapa_base.qml
# - Gera figura Plotly (retorna em `fig`); opcional: write_html

from __future__ import annotations
import re
import xml.etree.ElementTree as ET
from pathlib import Path
import pandas as pd
import plotly.graph_objects as go

# =========================
# caminho do QML
# =========================
qml_path = data_dir / "mapa_base.qml"  # ajuste se necessário

# =========================
# helpers de cor / QML
# =========================
def parse_qml_colorramp(qml_file: Path):
    """
    Lê QML (QGIS Raster Color Ramp Shader) e retorna lista de (valor_float, '#RRGGBB').
    Suporta <item value="..." color="r,g,b,a" .../> ou hex.
    """
    tree = ET.parse(qml_file)
    root = tree.getroot()
    items = root.findall(".//item")
    if not items:
        items = root.findall(".//colorramp//item")

    stops: list[tuple[float, str]] = []
    for it in items:
        val = float(it.attrib.get("value"))
        color_attr = it.attrib.get("color") or it.attrib.get("label", "")
        if "," in color_attr:
            parts = [int(x) for x in color_attr.split(",")]
            r, g, b = parts[:3]
            hex_color = f"#{r:02X}{g:02X}{b:02X}"
        else:
            hex_color = color_attr.strip()
            if not hex_color.startswith("#"):
                hex_color = "#" + hex_color
            if not re.fullmatch(r"#?[0-9A-Fa-f]{6}", hex_color):
                hex_color = "#000000"
        stops.append((val, hex_color))

    stops.sort(key=lambda x: x[0])
    return stops

def hex_to_rgb_tuple(hex_color: str) -> tuple[int, int, int]:
    hex_color = hex_color.lstrip("#")
    return tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))

def rgb_to_plotly_str(rgb_tuple: tuple[int, int, int]) -> str:
    r, g, b = rgb_tuple
    return f"rgb({r},{g},{b})"

def build_plotly_colorscale_from_qml(stops: list[tuple[float, str]]):
    vals = [v for v, _ in stops]
    vmin, vmax = min(vals), max(vals)
    span = vmax - vmin if vmax > vmin else 1.0
    colorscale = []
    for v, hex_color in stops:
        t = (v - vmin) / span
        colorscale.append([t, rgb_to_plotly_str(hex_to_rgb_tuple(hex_color))])
    return colorscale, vmin, vmax

def interpolate_hex_color(stops: list[tuple[float, str]], value: float) -> str:
    """Interpola a cor hex para `value` com base nos stops do QML (linear em RGB)."""
    stops = sorted(stops, key=lambda x: x[0])
    vmin, vmax = stops[0][0], stops[-1][0]
    if value <= vmin:
        return stops[0][1]
    if value >= vmax:
        return stops[-1][1]
    for i in range(len(stops) - 1):
        v0, c0 = stops[i]
        v1, c1 = stops[i+1]
        if v0 <= value <= v1:
            t = (value - v0) / (v1 - v0) if v1 > v0 else 0.0
            r0, g0, b0 = hex_to_rgb_tuple(c0)
            r1, g1, b1 = hex_to_rgb_tuple(c1)
            r = int(round(r0 + t*(r1 - r0)))
            g = int(round(g0 + t*(g1 - g0)))
            b = int(round(b0 + t*(b1 - b0)))
            return f"#{r:02X}{g:02X}{b:02X}"
    return stops[-1][1]

def ideal_text_color_for_bg(hex_color: str) -> str:
    """Retorna 'white' ou 'black' por luminância (WCAG) para garantir contraste."""
    r, g, b = [x/255.0 for x in hex_to_rgb_tuple(hex_color)]
    def f(c): 
        return c/12.92 if c <= 0.03928 else ((c+0.055)/1.055) ** 2.4
    L = 0.2126*f(r) + 0.7152*f(g) + 0.0722*f(b)
    return "black" if L > 0.6 else "white"

# =========================
# preparação dos dados
# (usa gdf_favelas já no ambiente)
# =========================
def preparar_top10(gdf_favelas) -> pd.DataFrame:
    df = gdf_favelas[[ "nm_fcu", "Pop_2022", "temp_media" ]].copy()
    df["Pop_2022"]  = pd.to_numeric(df["Pop_2022"], errors="coerce")
    df["temp_media"] = pd.to_numeric(df["temp_media"], errors="coerce")
    df = df.dropna(subset=["nm_fcu", "Pop_2022", "temp_media"])
    dfx = df.sort_values("Pop_2022", ascending=False).head(10).copy()
    # fixa a ordem visual no eixo x pela ordem do top10
    dfx["nm_fcu"] = pd.Categorical(dfx["nm_fcu"], categories=dfx["nm_fcu"].tolist(), ordered=True)
    return dfx

# =========================
# figura plotly
# =========================
def grafico_barras_pop_temp_horizontal(gdf_favelas, qml_path: Path,
                                       titulo: str = "Top 10 favelas por população"):
    # rampa do QML -> colorscale
    stops = parse_qml_colorramp(qml_path)
    colorscale, cmin, cmax = build_plotly_colorscale_from_qml(stops)

    dfx = preparar_top10(gdf_favelas)

    fig = go.Figure()
    fig.add_bar(
        x=dfx["Pop_2022"],
        y=dfx["nm_fcu"],
        orientation="h",
        marker=dict(
            color=dfx["temp_media"],
            colorscale=colorscale,
            cmin=cmin,
            cmax=cmax,
            colorbar=dict(title="Temperatura média (°C)", thickness=14)
        ),
        hovertemplate="<b>%{y}</b><br>População: %{x:,}<br>Temp. média: %{customdata:.2f} °C<extra></extra>",
        customdata=dfx["temp_media"].values,
    )

    # rótulos
    for y, pop, t in zip(dfx["nm_fcu"], dfx["Pop_2022"], dfx["temp_media"]):
        pop_str = f"{int(round(pop)):,}".replace(",", ".")
        # população no fim da barra
        fig.add_annotation(
            x=float(pop),
            y=y,
            text=pop_str,
            showarrow=False,
            xshift=6,
            font=dict(size=12, color="black"),
            xanchor="left",
            yanchor="middle"
        )
        # # temperatura no meio da barra
        # bar_hex = interpolate_hex_color(stops, float(t))
        # text_color = ideal_text_color_for_bg(bar_hex)
        # fig.add_annotation(
        #     x=float(pop)/2.0,
        #     y=y,
        #     text=f"{t:.1f}°C",
        #     showarrow=False,
        #     font=dict(size=12, color=text_color),
        #     xanchor="center",
        #     yanchor="middle"
        # )

    fig.update_layout(
        title=titulo,
        xaxis_title="População",
        yaxis_title="Favela",
        bargap=0.25,
        template="plotly_white",
        margin=dict(l=40, r=20, t=60, b=40),
        hovermode="y",
        yaxis=dict(
            categoryorder="array",
            categoryarray=dfx.sort_values("Pop_2022", ascending=True)["nm_fcu"].tolist()
        )
    )
    fig.update_xaxes(tickformat=",d")
    return fig

# uso:
fig = grafico_barras_pop_temp_horizontal(gdf_favelas, qml_path)
fig.show()  # descomente para visualizar no notebook
# fig.write_html("outputs/figures/top10_favelas_pop_temp.html", include_plotlyjs="cdn")  # opcional


In [15]:
# %% [markdown]
# Top 10 favelas mais quentes e mais frias (barras horizontais)
# - Comprimento da barra = temp_media (°C)
# - Cor = temp_media com rampa do QML (data/mapa_base.qml)
# - Hover mostra temp e Pop_2022
# - Ordenação com o "extremo" no topo
# - Eixo X comum (0 → teto arredondado em múltiplos de 5)

import numpy as np
import plotly.graph_objects as go

def _prep_top10_por_temp(gdf_favelas: pd.DataFrame, mais_quentes: bool) -> pd.DataFrame:
    df = gdf_favelas[["nm_fcu", "Pop_2022", "temp_media"]].copy()
    df["Pop_2022"] = pd.to_numeric(df["Pop_2022"], errors="coerce")
    df["temp_media"] = pd.to_numeric(df["temp_media"], errors="coerce")
    df = df.dropna(subset=["nm_fcu", "Pop_2022", "temp_media"])

    if mais_quentes:
        dfx = df.sort_values("temp_media", ascending=False).head(10).copy()
    else:
        dfx = df.sort_values("temp_media", ascending=True).head(10).copy()
    return dfx

def grafico_top10_por_temperatura(
    gdf_favelas,
    qml_path: Path,
    mais_quentes: bool = True,
    temp_max_global: float | None = None
):
    # rampa do QML
    stops = parse_qml_colorramp(qml_path)
    colorscale, cmin, cmax = build_plotly_colorscale_from_qml(stops)

    dfx = _prep_top10_por_temp(gdf_favelas, mais_quentes)
    titulo = "Top 10 favelas mais quentes" if mais_quentes else "Top 10 favelas mais frias"

    fig = go.Figure()
    fig.add_bar(
        x=dfx["temp_media"],
        y=dfx["nm_fcu"],
        orientation="h",
        marker=dict(
            color=dfx["temp_media"],
            colorscale=colorscale,
            cmin=cmin,
            cmax=cmax,
            colorbar=dict(title="Temperatura média (°C)", thickness=14)
        ),
        hovertemplate="<b>%{y}</b><br>Temp. média: %{x:.2f} °C<br>População: %{customdata:,}<extra></extra>",
        customdata=dfx["Pop_2022"].values,
    )

    # rótulos de temperatura no fim da barra
    for y, temp in zip(dfx["nm_fcu"], dfx["temp_media"]):
        fig.add_annotation(
            x=float(temp),
            y=y,
            text=f"{temp:.1f}°C",
            showarrow=False,
            xshift=6,
            font=dict(size=12, color="black"),
            xanchor="left",
            yanchor="middle"
        )

    fig.update_layout(
        title=titulo,
        xaxis_title="Temperatura média (°C)",
        yaxis_title="Favela",
        bargap=0.25,
        template="plotly_white",
        margin=dict(l=40, r=20, t=60, b=40),
        hovermode="y",
    )

    # ORDEM DO EIXO Y (extremo no topo)
    if mais_quentes:
        order = dfx.sort_values("temp_media", ascending=True)["nm_fcu"].tolist()
    else:
        order = dfx.sort_values("temp_media", ascending=False)["nm_fcu"].tolist()
    fig.update_yaxes(categoryorder="array", categoryarray=order)

    # ESCALA GLOBAL (0 → teto comum, arredondado para múltiplos de 5)
    if temp_max_global is None:
        temp_series = pd.to_numeric(gdf_favelas["temp_media"], errors="coerce")
        temp_max_global = float(np.nanmax(temp_series.values))
    temp_max_global = int(np.ceil(temp_max_global / 5.0) * 5.0)
    fig.update_xaxes(range=[0, temp_max_global], dtick=5, tick0=0)

    return fig

# === calcular teto global "bonito" uma única vez (opcional, mas explícito) ===
_temp_series = pd.to_numeric(gdf_favelas["temp_media"], errors="coerce")
_temp_max_observado = float(np.nanmax(_temp_series.values))
temp_max_global_nice = int(np.ceil(_temp_max_observado / 5.0) * 5.0)

# === gerar as figuras já usando o mesmo teto ===
fig_quentes = grafico_top10_por_temperatura(
    gdf_favelas, qml_path, mais_quentes=True,  temp_max_global=temp_max_global_nice
)
fig_frias   = grafico_top10_por_temperatura(
    gdf_favelas, qml_path, mais_quentes=False, temp_max_global=temp_max_global_nice
)

fig_quentes.show()
fig_frias.show()

# (opcional) salvar
# out_dir = Path("outputs/figures"); out_dir.mkdir(parents=True, exist_ok=True)
# fig_quentes.write_html(out_dir / "top10_favelas_mais_quentes.html", include_plotlyjs="cdn")
# fig_frias.write_html(out_dir / "top10_favelas_mais_frias.html", include_plotlyjs="cdn")
